# Fine-Tuning QA model

https://www.youtube.com/watch?v=3XiJrn_8F9Q&t=1197s <br>
https://www.youtube.com/watch?v=6ppxd9lp-X0


In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Aug 27 18:35:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 122.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 27.2 MB/s eta 0:00:00


In [4]:
!pip install accelerate -U

In [5]:
!pip show accelerate

Name: accelerate
Version: 0.22.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: sylvain@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, packaging, psutil, pyyaml, torch
Required-by: 


In [6]:
import torch
torch.cuda.is_available()

True

let us see first the capability of the pretrained deepset/robert-base-squad2 model on our questions

In [10]:
import torch
from transformers import RobertaTokenizer, RobertaForQuestionAnswering

# Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("deepset/roberta-base-squad2")
model = RobertaForQuestionAnswering.from_pretrained("deepset/roberta-base-squad2")

# Read context from a .txt file
import requests

url = "https://raw.githubusercontent.com/AlinZohari/InformationExtraction/main/data/authorize_doc/Kuiper_FCC-20-102A1.txt"
response = requests.get(url)
context = response.text

# Dictionary of questions
questions = {
    "const_name": "What's the name of the satellite constellation the company seeks to deploy or operate?",
    "date_release": "On which date was the document released?",
    "date_50": "By which date must the company launch and operate half of its satellites?",
    "date_100": "By which date is the company expected to have all its satellites operational?",
    "total_sat_const": "How many satellites is the company authorized to deploy and operate for this constellation?",
    "altitude": "At which authorized altitudes will the company deploy its satellites?",
    "inclination": "What are the authorized satellite inclinations within the corresponding altitudes?",
    "number_orb_plane": "How many orbital planes, corresponding to given altitudes and inclinations, has the company been authorized for?",
    "total_sat_per_orb_plane": "How many satellites are allocated to each orbital plane?",
    "total_sat_per_alt_incl": "How many satellites, for each altitude and inclination, are there across all matching orbital planes?",
    "operational_lifetime": "What is the satellite's expected operational lifetime in years?"
}

# Loop through each question
for key, question in questions.items():
    # Prepare the input
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)


    # Get the model's prediction
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    output = model(input_ids, attention_mask=attention_mask)

    answer_start_scores = output.start_logits
    answer_end_scores = output.end_logits

    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores)
    answer = tokenizer.decode(input_ids[0][answer_start:answer_end + 1])

    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print()


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Question: What's the name of the satellite constellation the company seeks to deploy or operate?
Answer:  NGSO FSS



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Question: On which date was the document released?
Answer:  July 30, 2020



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


...

Question: How many orbital planes, corresponding to given altitudes and inclinations, has the company been authorized for?
Answer: <s>



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Question: How many satellites are allocated to each orbital plane?
Answer: <s>



Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Question: How many satellites, for each altitude and inclination, are there across all matching orbital planes?
Answer: <s>

Question: What is the satellite's expected operational lifetime in years?
Answer: <s>



The warning message you are seeing is due to the truncation strategy used by the tokenizer. The 'longest_first' truncation strategy truncates tokens from the longest of the two sequences (question or context) until they fit within the specified max_length. The warning is informing you that the overflowing tokens, which are the tokens removed during truncation, are not being returned in the inputs. This is expected behavior, as we are not using the overflowing tokens in this case.

The answers that are just indicate that the model is not able to find a suitable answer in the context for the given question. This could be because the answer is not present in the context, or because the context is too large and the relevant portion was truncated.

Because of this let us fine tune this model to fit our purpose.

In [11]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

#Reference: https://huggingface.co/deepset/roberta-base-squad2

model_name = "deepset/roberta-base-squad2"

#Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [12]:
#looking at RoBerta Question Answering
model

RobertaForQuestionAnswering(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Lay

How to fine-tune a QA model
- we need GPU
- building a training script


In [13]:
import requests
import json

url = "https://raw.githubusercontent.com/AlinZohari/InformationExtraction/main/data/QA_model/train.json"
response = requests.get(url)
train = response.json()

In [14]:
#looking at the train dataset
train

[{'context': 'In this Order and Authorization, we grant, to the extent set forth below, the request of Kuiper Systems LLC (Kuiper or Amazon) to provide satellite services.\n            Operating 3,372 satellites in 102 orbital planes at altitudes of 590 km, 610 km, and 630 km in a circular orbit.\n            At 590 km, 30 orbital planes with 28 satellites per plane for a total of 840 satellites at inclination of 33 degree.\n            At 610 km, 42 orbital planes with 36 satellites per plane for a total of 1512 satellites at inclination of 42 degree.\n            At 630 km, 30 orbital planes with 34 satellites per plane for a total of 1020 satellite at inclination of 51.9 degree.\n            The constellation are require to launch and operate 50 percent of its satellites no later than July 30, 2026, and must launch the remaining space stations necessary to complete its authorized service constellation, place them in their assigned orbits, and operate each of them in accordance with 

In [15]:
import requests
import json

url = "https://raw.githubusercontent.com/AlinZohari/InformationExtraction/main/data/QA_model/validation.json"
response = requests.get(url)
validation = response.json()

In [16]:
#looking atthe validation dataset
validation

[{'context': 'Release date: October 29, 1995 In this Order and Authorization, we grant, to the extent set forth below, the request of Ligado Networks LLC to provide Fixed Satellite Services (FSS). Operating 2320 satellites in 58 orbital planes in total at altitudes of 500, 600, 700 and 800 kilometers. At an altitude of 500 km, there are 15 orbital planes, each hosting 36 satellites, resulting in a total of 540 satellites at an inclination of 36.5 degrees. For the 600 km altitude, 23 orbital planes are present, with each plane containing 50 satellites, summing up to 1150 satellites at an inclination of 49 degrees. At the 700 km mark, 15 orbital planes are equipped with 27 satellites each, leading to a total of 405 satellites at a 51.9-degree inclination. Lastly, at 800 km, there are 5 orbital planes, and each has 45 satellites, amounting to a total of 225 satellites at an inclination of 59.3 degrees. The constellation are require to launch and operate 50 percent of its satellites no lat

## Preprocess the data

In [ ]:
!pip install datasets

In [33]:
import pandas as pd
from datasets import Dataset

def preprocess_function(examples):
    questions = []
    contexts = []
    answers = []

    for i in range(len(examples['context'])):
        context = examples['context'][i]
        qas = examples['qas'][i]

        for qa in qas:
            questions.append(qa['question'].strip())
            contexts.append(context)
            if not qa['is_impossible']:
                ans = qa['answers'][0]  # Assuming each question has only one correct answer
                answers.append({'answer_start': [ans['answer_start']], 'text': [ans['text']]})
            else:
                answers.append({'answer_start': [0], 'text': ['']})

    inputs = tokenizer(
        questions,
        contexts,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")

    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer['answer_start'][0]
        end_char = start_char + len(answer['text'][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

# Convert lists to Dataset objects
train_dataset = Dataset.from_pandas(pd.DataFrame(train))
validation_dataset = Dataset.from_pandas(pd.DataFrame(validation))

# Apply preprocess_function
tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_validation = validation_dataset.map(preprocess_function, batched=True, remove_columns=validation_dataset.column_names)



Map:   0%|          | 0/13 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

The DefaultDataCollator is a class from the transformers library that is used to collate samples into batches for training or evaluation. When you train a model, you usually don't pass the entire dataset at once, but rather use mini-batches of data. The data_collator is responsible for taking the individual samples and combining them into these mini-batches.

The DefaultDataCollator will:

Handle the padding of the input data (if necessary) to ensure that all samples in the batch have the same length.
Convert the batch into PyTorch tensors.

In [34]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

## Training

In [35]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [ ]:
#defining training argument
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

#Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_validation,
    data_collator=data_collator,
)


trainer.train()